In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import uniform
import os
from os import listdir, getcwd
import re
import sys
import warnings
import sklearn
import math
from scipy import stats
import openpyxl
from IPython.display import clear_output

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, LeaveOneGroupOut, StratifiedKFold
from sklearn.feature_selection import VarianceThreshold, SelectFpr, SelectFwe, RFECV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.externals import joblib
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.linear_model import ElasticNetCV, LogisticRegression, SGDClassifier, LogisticRegressionCV, LarsCV, LassoLarsCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

### IMPORT DATASET 
##### ONLY LMCI/EMCI SUBJECTS; 
##### BASELINE INFORMATION + CONVERSION AT 3-YEARS 

In [2]:
dataset = pd.read_excel("../../adnimerge_screen.xlsx")

In [3]:
variables_to_scree_out = ["PTID", "VISCODE", "COLPROT", "ORIGPROT", "EXAMDATE", "PTETHCAT", "PTRACCAT", "FDG", "PIB", "AV45", "ABETA", "TAU", "PTAU", "EcogPtMem", "MOCA","EcogPtLang", "EcogPtVisspat", "EcogPtPlan", "EcogPtOrgan", "EcogPtDivatt", "EcogPtTotal", "EcogSPMem", "EcogSPLang", "EcogSPVisspat", "EcogSPPlan", "EcogSPOrgan", "EcogSPDivatt", "EcogSPTotal", "FLDSTRENG", "FSVERSION", "Ventricles", "Hippocampus", "WholeBrain", "Entorhinal", "Fusiform", "MidTemp", "ICV", "mPACCdigit", "mPACCtrailsB", "EXAMDATE_bl", "CDRSB_bl", "ADAS11_bl", "ADAS13_bl", "ADASQ4_bl", "MMSE_bl", "RAVLT_immediate_bl", "RAVLT_learning_bl", "RAVLT_forgetting_bl", "RAVLT_perc_forgetting_bl", "FAQ_bl", "mPACCdigit_bl", "mPACCtrailsB_bl", "FLDSTRENG_bl", "FSVERSION_bl", "Ventricles_bl", "Hippocampus_bl", "WholeBrain_bl", "Entorhinal_bl", "Fusiform_bl", "MidTemp_bl", "ICV_bl", "MOCA_bl", "EcogPtMem_bl", "EcogPtLang_bl", "EcogPtVisspat_bl", "EcogPtPlan_bl", "EcogPtOrgan_bl", "EcogPtDivatt_bl", "EcogPtTotal_bl", "EcogSPMem_bl", "EcogSPLang_bl", "EcogSPVisspat_bl", "EcogSPPlan_bl", "EcogSPOrgan_bl", "EcogSPDivatt_bl", "EcogSPTotal_bl", "ABETA_bl", "TAU_bl", "PTAU_bl", "FDG_bl", "PIB_bl", "AV45_bl", "Years_bl", "Month_bl", "Month", "M", "DX","update_stamp"]
dataset = dataset.drop(variables_to_scree_out, axis=1)

In [4]:
# SHOWING A PREVIEW OF THE DATASET

display(dataset.iloc[0:10,:])

,RID,SITE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTMARRY,APOE4,CDRSB,ADAS11,...,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,CONVERSION_AT_3Y
0,4,22,LMCI,67.5,Male,10,Married,0,1.0,14.33,...,27,37,7,4,36.3636,4,25.0,271.0,0.0,NO
1,6,100,LMCI,80.4,Female,13,Married,0,0.5,18.67,...,25,30,1,5,83.3333,3,34.0,168.0,0.0,NO
2,42,23,LMCI,72.8,Male,18,Married,0,0.5,7.00,...,30,29,6,8,88.8889,1,47.0,101.0,2.0,YES
3,51,99,LMCI,66.5,Male,18,Married,2,1.0,9.67,...,27,29,1,4,57.1429,7,50.0,94.0,2.0,NO
4,54,99,LMCI,81.0,Female,20,Married,0,2.5,20.33,...,27,26,3,6,100.0000,0,20.0,201.0,12.0,YES
5,57,18,LMCI,77.3,Male,20,Married,1,1.5,15.67,...,27,29,6,8,100.0000,3,20.0,151.0,7.0,YES
6,77,67,LMCI,79.7,Male,18,Married,0,0.5,19.00,...,28,18,3,5,100.0000,1,35.0,157.0,3.0,YES
7,80,18,LMCI,85.0,Male,18,Married,1,1.5,14.00,...,27,23,4,3,42.8571,1,48.0,81.0,1.0,NO
8,98,67,LMCI,84.4,Female,16,Divorced,1,2.0,10.67,...,26,25,4,5,55.5556,7,38.0,167.0,4.0,YES
9,101,7,LMCI,73.6,Male,18,Married,2,0.5,7.00,...,27,24,3,5,71.4286,8,49.0,50.0,0.0,YES


In [5]:
display(dataset.describe())

,RID,SITE,AGE,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ
count,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,549.000000,547.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,260.000000,546.000000,546.000000
mean,2385.623636,74.014545,73.052545,16.018182,0.640000,1.509091,10.198506,16.460037,5.523636,27.594545,34.330909,4.092727,4.665455,60.799743,5.676364,37.653846,115.073260,3.214286
std,1775.001570,95.606280,7.353912,2.778337,0.683823,0.884448,4.453091,6.721507,2.558315,1.800922,10.420990,2.587018,2.433241,32.603198,3.412287,11.024165,65.100051,4.191719
min,4.000000,2.000000,55.000000,6.000000,0.000000,0.500000,2.000000,3.000000,0.000000,23.000000,11.000000,-1.000000,-2.000000,-25.000000,0.000000,5.000000,33.000000,0.000000
25%,782.250000,23.000000,68.100000,14.000000,0.000000,1.000000,7.000000,11.000000,4.000000,26.000000,27.000000,2.000000,3.000000,33.333300,3.000000,30.000000,71.000000,0.000000
50%,2073.500000,57.000000,73.450000,16.000000,1.000000,1.500000,10.000000,16.000000,5.000000,28.000000,33.000000,4.000000,5.000000,62.500000,6.000000,38.000000,96.000000,2.000000
75%,4376.500000,123.000000,78.775000,18.000000,1.000000,2.000000,13.000000,21.000000,7.000000,29.000000,41.000000,6.000000,6.000000,100.000000,9.000000,45.000000,128.000000,5.000000
max,5066.000000,941.000000,88.300000,20.000000,2.000000,5.500000,27.670000,39.670000,10.000000,30.000000,68.000000,11.000000,13.000000,100.000000,15.000000,69.000000,300.000000,22.000000


### RECODE CATEGORICAL VARIABLES WITH NUMBERS (SKLEARN DOES NOT ACCECPT ANY CATEGORICAL VARIABLE EVEN FOR MODELS WHICH NATIVELY DO IT)

In [6]:
# CATEGORICAL COLUMS ARE [2,4,6,21]

# 2 - DX_bl
DX_bl = {'EMCI': 0,'LMCI': 1}
dataset.DX_bl = [DX_bl[item] for item in dataset.DX_bl]

# 4 - PTGENDER
PTGENDER = {'Male': 0,'Female': 1}
dataset.PTGENDER = [PTGENDER[item] for item in dataset.PTGENDER]

# 6 - PTMARRY
PTMARRY = {'Never married':0, u'Married': 1, 'Divorced': 2, 'Widowed': 3, 'Unknown':4 }
dataset.PTMARRY = [PTMARRY[item] for item in dataset.PTMARRY]
dataset.PTMARRY = dataset.PTMARRY.astype("int")

# 20 - CONVERSION_AT_3Y
CONVERSION_AT_3Y = {'NO': 0,'YES': 1}
dataset.CONVERSION_AT_3Y = [CONVERSION_AT_3Y[item] for item in dataset.CONVERSION_AT_3Y]

### IDENTIFICATION OF VARIABLES WITH TOO MANY MISSING VALUES

In [7]:
# REPLACE "UNKNOWN" IN PTMARRY AS MISSING VALUE
dataset.PTMARRY[dataset.PTMARRY == 4] = None

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
# AUTOMATICALLY REMOVE VARIABLES WITH MORE THAN A CERTAIN RATE OF MISSING VALUES

# THE DROP RATE
drop_rate = 0.2

# LOOP OVER AND IDENTIFY VARIABLES THAT NEED TO BE DROPPED
predictor_to_drop = list()
for j in range(dataset.shape[1]):
    print("MISSING VALUES IN '"+dataset.columns[j]+"' ARE: "+str(sum(dataset.iloc[:,j].isnull()))+" - "+str(round(sum(dataset.iloc[:,j].isnull())/dataset.shape[0]*100, 2))+"%")
    
    if sum(dataset.iloc[:,j].isnull()) > dataset.shape[0]*drop_rate:
        predictor_to_drop.append(dataset.columns[j])
        print("---------------------------> MISSING VALUES EXCEEDING THE CHOSEN DROP RATE OF "+str(drop_rate))
        print("---------------------------> VARIABLE '"+dataset.columns[j]+"' WILL BE REMOVED")

# DROP THE VARIABLES FROM THE DATASET
dataset = dataset.drop(columns = predictor_to_drop)
del predictor_to_drop

MISSING VALUES IN 'RID' ARE: 0 - 0.0%
MISSING VALUES IN 'SITE' ARE: 0 - 0.0%
MISSING VALUES IN 'DX_bl' ARE: 0 - 0.0%
MISSING VALUES IN 'AGE' ARE: 0 - 0.0%
MISSING VALUES IN 'PTGENDER' ARE: 0 - 0.0%
MISSING VALUES IN 'PTEDUCAT' ARE: 0 - 0.0%
MISSING VALUES IN 'PTMARRY' ARE: 3 - 0.55%
MISSING VALUES IN 'APOE4' ARE: 0 - 0.0%
MISSING VALUES IN 'CDRSB' ARE: 0 - 0.0%
MISSING VALUES IN 'ADAS11' ARE: 1 - 0.18%
MISSING VALUES IN 'ADAS13' ARE: 3 - 0.55%
MISSING VALUES IN 'ADASQ4' ARE: 0 - 0.0%
MISSING VALUES IN 'MMSE' ARE: 0 - 0.0%
MISSING VALUES IN 'RAVLT_immediate' ARE: 0 - 0.0%
MISSING VALUES IN 'RAVLT_learning' ARE: 0 - 0.0%
MISSING VALUES IN 'RAVLT_forgetting' ARE: 0 - 0.0%
MISSING VALUES IN 'RAVLT_perc_forgetting' ARE: 0 - 0.0%
MISSING VALUES IN 'LDELTOTAL' ARE: 0 - 0.0%
MISSING VALUES IN 'DIGITSCOR' ARE: 290 - 52.73%
---------------------------> MISSING VALUES EXCEEDING THE CHOSEN DROP RATE OF 0.2
---------------------------> VARIABLE 'DIGITSCOR' WILL BE REMOVED
MISSING VALUES IN 'TRABSCO

### CREATE PREDICTORS AND OUTCOME ARRAYS, AND SPLIT THEM IN TRAIN AND TEST DATASETS

In [9]:
# KEEPING TRACK OF COLUMN NAMES, AS WHEN THE DATASET IS CONVERTED IN np.array THE NAMES ARE LOST
# LAST COLUMN IS THE OUTCOME VARIABLE, SO IT IS EXCLUDED. RID (index 0) AND SITE (index 1) ARE JUST ID VARIABLES.

predictor_names = dataset.columns[0:(dataset.shape[1]-1)]

In [10]:
# CREATE PREDICOTRS AND OUTCOME ARRAYS

# The outcome variable is CONVERSION_AT_3Y, the last column
X = dataset.iloc[:,0:(dataset.shape[1]-1)].values
y = dataset["CONVERSION_AT_3Y"].values


################################
## SITE-INDEPENDENT TEST FOLD ##
################################

# split in train and test set, holding-out out for testing the subjects recruited in the sites allocated to the current test fold

# read the current fold from the current working directory and assing the correct sites for this test fold
fold = [int(os.getcwd()[-1])][0]

if fold == 1:
    SITE_test = [6,12,18,21,126,127,128,137]
elif fold == 2:
    SITE_test = [2,3,9,23,24,29,36,37,94,99,114]
elif fold == 3:
    SITE_test = [7,13,14,33,41,67,73,98,100,109,116]
elif fold == 4:
    SITE_test = [5,16,22,27,31,35,123,130,141,153]
elif fold == 5:
    SITE_test = [10,11,19,32,51,52,53,57,62,68,72,82,129,131,133,135,136,941]
    
# CREATE THE TRAIN/TEST SPLIT

X_train = X[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in SITE_test],:]
X_test = X[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in SITE_test],:]

y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in SITE_test]]
y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in SITE_test]]


# CHECK IF THE TRAIN AND TEST SET ARE APPROXIMATELY STRATIFIED FOR THE OUTCOME AND APPROXIMATELY 15% OF CASES HAV BEEN ASSIGNET TO THE TEST SET

print("% of cases hold-out for the test set is:")
print(y_train.shape[0]/y.shape[0])

print("____________________________")

print("N cases in train set: "+str(y_train.shape[0]))
print("% of CONVERTERS in the train set is: "+str(y_train[y_train == 1].shape[0]/y_train.shape[0]))
print("% of NON- CONVERTERS in the train set is: "+str(y_train[y_train == 0].shape[0]/y_train.shape[0]))


print("N cases in train set: "+str(y_test.shape[0]))
print("% of CONVERTERS in the train set is:"+str(y_test[y_test == 1].shape[0]/y_test.shape[0]))
print("% of NON- CONVERTERS in the train set is:"+str(y_test[y_test == 0].shape[0]/y_test.shape[0]))

% of cases hold-out for the test set is:
0.8018181818181818
____________________________
N cases in train set: 441
% of CONVERTERS in the train set is: 0.35827664399092973
% of NON- CONVERTERS in the train set is: 0.6417233560090703
N cases in train set: 109
% of CONVERTERS in the train set is:0.3577981651376147
% of NON- CONVERTERS in the train set is:0.6422018348623854


### MISSING VALUES IN THE PREDICTORS OF BOTH TRAIN/TEST DATASET ARE INPUTED WITH THE MEDIAN OF THE TRAIN DATASET

In [11]:
median_imputation = np.zeros(X_train.shape[1])
for j in range(X_train.shape[1]):
    median_imputation[j] = np.nanmedian(X_train[:,j])
    X_train[np.where(np.isnan(X_train[:,j]) == True),j] = median_imputation[j]
    X_test[np.where(np.isnan(X_test[:,j]) == True),j] = median_imputation[j]

### RECODE POLYTOMOUS CATEGORICAL PREDICTOR WITH ONE-HOT ENCODING

In [12]:
# PTMARRY (index 6)
enc = OneHotEncoder(categorical_features=[6], sparse=False)
enc.fit(X_train)   

X_train = enc.transform(X_train)
X_test = enc.transform(X_test)

# APOE4 (index 11)
enc = OneHotEncoder(categorical_features=[10], sparse=False)
enc.fit(X_train)   

# TRANFORM THE DATASET
X_train = enc.transform(X_train)
X_test = enc.transform(X_test)

# UPDATE THE PREDICOTOR NAMES LIST

# removing 'PTMARRY'and 'APOE4' from the name of predictors using list comprehension
predictor_names = [x for x in predictor_names if x!='PTMARRY']
predictor_names = [x for x in predictor_names if x!='APOE4']

# names of the new one-hot predictors
new_predictor = ['APOE4_0','APOE4_1','APOE4_2','PTMARRY_Never married', 'PTMARRY_Married', 'PTMARRY_Divorced', 'PTMARRY_Widowed']

# make the new correct list of predictor names
predictor_names = new_predictor + predictor_names

### STANDARDIZE CONTINUOS PREDICTORS (MEAN=0; SD=1)

In [13]:
# CONTINUOS PREDICTORS ARE [[11,14:X_train.shape[1]]
features_to_scale = [x for x in [range(14,X_train.shape[1])]]
features_to_scale = np.append([11],features_to_scale)

# CREATE THE SCALER, TRAINED IN X_train
scaler = StandardScaler()
scaler.fit(X_train[:,features_to_scale])

# TRANSFORM BOTH X_traiN AND X_test
X_train[:,features_to_scale] = scaler.transform(X_train[:,features_to_scale])
X_test[:,features_to_scale] = scaler.transform(X_test[:,features_to_scale])


# CORRELATED PREDICTORS

### Investigate if there exists in the train set very correlated features (r >.75)

In [14]:
# define function to calculate the correlation matrix

from scipy.stats import pearsonr, spearmanr

def corrcoef_loop(matrix):
    rows, cols = matrix.shape[0], matrix.shape[1]
    r = np.ones(shape=(rows, rows))
    p = np.ones(shape=(rows, rows))
    for i in range(rows):
        for j in range(i+1, rows):
            r_, p_ = pearsonr(matrix[i], matrix[j])
            r[i, j] = r[j, i] = r_
            p[i, j] = p[j, i] = p_
    return r, p

# Calculate and print the correlation matrix for the training dataset

matrix = corrcoef_loop(np.transpose(X_train))
matrix = pd.DataFrame(np.around(matrix[0],2))
matrix.index = predictor_names
matrix.columns = predictor_names
display(matrix)

# Highlight couples of predictors above the define threshold

display(np.where(np.abs(matrix) > 0.75))

,APOE4_0,APOE4_1,APOE4_2,PTMARRY_Never married,PTMARRY_Married,PTMARRY_Divorced,PTMARRY_Widowed,RID,SITE,DX_bl,...,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,TRABSCOR,FAQ
APOE4_0,1.00,-0.79,-0.34,0.06,-0.08,-0.03,0.11,0.02,0.07,-0.07,...,-0.25,-0.24,0.16,0.15,0.14,-0.15,-0.24,0.17,-0.04,-0.17
APOE4_1,-0.79,1.00,-0.31,-0.03,0.05,0.02,-0.07,-0.03,-0.02,0.06,...,0.16,0.14,-0.09,-0.10,-0.08,0.10,0.16,-0.11,0.09,0.17
APOE4_2,-0.34,-0.31,1.00,-0.05,0.05,0.02,-0.06,0.02,-0.07,0.03,...,0.14,0.15,-0.10,-0.08,-0.11,0.07,0.12,-0.09,-0.07,0.01
PTMARRY_Never married,0.06,-0.03,-0.05,1.00,-0.23,-0.04,-0.05,0.07,0.17,-0.10,...,-0.04,-0.02,0.04,-0.07,-0.02,-0.05,-0.02,0.10,0.00,-0.04
PTMARRY_Married,-0.08,0.05,0.05,-0.23,1.00,-0.61,-0.66,-0.08,-0.02,0.04,...,0.12,0.09,0.00,-0.09,-0.11,-0.02,0.07,-0.05,0.03,0.12
PTMARRY_Divorced,-0.03,0.02,0.02,-0.04,-0.61,1.00,-0.12,0.10,-0.02,-0.08,...,-0.17,-0.13,0.01,0.13,0.10,0.03,-0.08,0.12,-0.16,-0.09
PTMARRY_Widowed,0.11,-0.07,-0.06,-0.05,-0.66,-0.12,1.00,-0.02,-0.02,0.06,...,0.01,0.01,-0.03,0.02,0.06,0.02,-0.01,-0.08,0.10,-0.06
RID,0.02,-0.03,0.02,0.07,-0.08,0.10,-0.02,1.00,0.04,-0.49,...,-0.14,-0.14,0.19,0.19,0.17,0.03,-0.08,0.34,-0.12,-0.09
SITE,0.07,-0.02,-0.07,0.17,-0.02,-0.02,-0.02,0.04,1.00,-0.05,...,-0.07,-0.09,0.06,0.05,-0.00,-0.10,-0.10,0.08,-0.02,-0.04
DX_bl,-0.07,0.06,0.03,-0.10,0.04,-0.08,0.06,-0.49,-0.05,1.00,...,0.38,0.38,-0.27,-0.36,-0.31,0.08,0.29,-0.71,0.19,0.23


(array([ 0,  0,  1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
        14, 15, 15, 15, 16, 16, 17, 18, 19, 20, 20, 21, 21, 22, 23, 24]),
 array([ 0,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
        15, 14, 15, 16, 15, 16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24]))

### Calculate components and substitute them in the dataset

In [15]:
# Fit the PCA

from sklearn.decomposition import PCA

PCA_corr_1 = PCA(copy=False)
PCA_corr_2 = PCA(copy=False)

PCA_corr_1.fit(X_train[:,[14,15,16]])
PCA_corr_2.fit(X_train[:,[20,21]])

print("Explained variance of the components of the first group of predictors")
print(PCA_corr_1.explained_variance_)
print("Number of components with explained variance > 1.0: "+ str(len(np.where(PCA_corr_1.explained_variance_ > 1))))
print(" ")
print("Explained variance of the components of the second group of predictors")
print(PCA_corr_2.explained_variance_)
print("Number of components with explained variance > 1.0: "+ str(len(np.where(PCA_corr_2.explained_variance_ > 1))))

Explained variance of the components of the first group of predictors
[ 2.63507959  0.35770888  0.01402971]
Number of components with explained variance > 1.0: 1
 
Explained variance of the components of the second group of predictors
[ 1.78663571  0.21790974]
Number of components with explained variance > 1.0: 1


In [16]:
# calculate the components

f_1_train = PCA_corr_1.transform(X_train[:,[14,15,16]])[:,0]
f_2_train = PCA_corr_2.transform(X_train[:,[20,21]])[:,0]

f_1_test = PCA_corr_1.transform(X_test[:,[14,15,16]])[:,0]
f_2_test = PCA_corr_2.transform(X_test[:,[20,21]])[:,0]

f_1_train = f_1_train.reshape((f_1_train.shape[0],1))
f_2_train = f_2_train.reshape((f_2_train.shape[0],1))

f_1_test = f_1_test.reshape((f_1_test.shape[0],1))
f_2_test = f_2_test.reshape((f_2_test.shape[0],1))

In [17]:
# substitute the correlated predicotrs in the datasets with their first PCs

features_to_keep = [i for i in range(X_train.shape[1]) if i not in [14,15,16,20,21]]

X_train = np.append(X_train[:,features_to_keep],f_1_train, axis=1)
X_train = np.append(X_train,f_2_train, axis=1)

X_test = np.append(X_test[:,features_to_keep],f_1_test, axis=1)
X_test = np.append(X_test,f_2_test, axis=1)

In [18]:
# update the predictor_names vector

predictor_names = [predictor_names[i] for i in features_to_keep]
predictor_names = np.append(np.append(predictor_names,"PCA_1_ADAS"),"PCA_1_RAVLT_forgetting").tolist()

# FEATURE SELECTION

### FEATURE_SET_1: ALL FEATURES - EXCLUDING APOE4

In [19]:
# ID ['RID', 'SITE'] + APOE4 ['APOE4_0', 'APOE4_1', 'APOE4_2'] FEATURES ARE EXCLUDED

# CREATE AN ARRAY WITH INDEXES OF FEATURES TO BE USED IN THE CURRENT FEATURE SET
feature_set_1 = [list(range(X_train.shape[1]))[i] for i in range(X_train.shape[1]) if predictor_names[i] not in ['RID', 'SITE','APOE4_0', 'APOE4_1', 'APOE4_2']]
print([predictor_names[i] for i in feature_set_1])

['PTMARRY_Never married', 'PTMARRY_Married', 'PTMARRY_Divorced', 'PTMARRY_Widowed', 'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'LDELTOTAL', 'TRABSCOR', 'FAQ', 'PCA_1_ADAS', 'PCA_1_RAVLT_forgetting']


### FEATURE_SET_2: FILTERING P<0.05 - EXCLUDING APOE4

In [20]:
# CREATE AND FIT THE FILTER
selector_feature_set_2 = SelectFwe(alpha=0.05)
selector_feature_set_2.fit(X_train, y=y_train)

# CREATE AN ARRAY WITH INDEXES OF FEATURES TO BE USED IN THE CURRENT FEATURE SET ACCORDING TO THE SELECTION STRATEGY
feature_set_2 = [list(range(X_train.shape[1]))[i] for i in range(X_train.shape[1]) if selector_feature_set_2.pvalues_[i] < 0.05 and predictor_names[i] not in ['RID', 'SITE','APOE4_0', 'APOE4_1', 'APOE4_2']]
print([predictor_names[i] for i in feature_set_2])

['DX_bl', 'AGE', 'CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'LDELTOTAL', 'TRABSCOR', 'FAQ', 'PCA_1_ADAS', 'PCA_1_RAVLT_forgetting']


### FEATURE_SET_5: WRAPPER (RFE) WITH LOGISTIC REGRESSION - EXCLUDING APOE4

In [21]:
# DEFINE CROSS-VALIDATION ITERATOR (THE SAME AS IT WILL BE DEFINED LATER DURING THE TRAINING THE MODELS)
n_fold = 10
n_repeats = 10
cv_random_seed = 1
rkf = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=n_repeats, random_state=cv_random_seed)

# CREATE AND FIT THE FILTER
selector_feature_set_5 = RFECV(estimator=LogisticRegression(penalty = 'l2', C = sys.maxsize), \
                             cv=rkf, \
                             scoring = "roc_auc",\
                             n_jobs = -1)
selector_feature_set_5.fit(X_train[:,feature_set_1], y=y_train)

# CREATE AN ARRAY WITH INDEXES OF FEATURES TO BE USED IN THE CURRENT FEATURE SET ACCORDING TO THE SELECTION STRATEGY
feature_set_5 = [feature_set_1[i] for i in range(len(selector_feature_set_5.support_)) if selector_feature_set_5.support_[i] == True ]
print([predictor_names[i] for i in feature_set_5])

['PTMARRY_Never married', 'PTMARRY_Married', 'PTMARRY_Divorced', 'PTMARRY_Widowed', 'DX_bl', 'PTGENDER', 'CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'LDELTOTAL', 'TRABSCOR', 'FAQ', 'PCA_1_ADAS', 'PCA_1_RAVLT_forgetting']


### FEATURE_SET_7: WRAPPER (RFE) WITH RANDOM FOREST WITH DEFAULT HYPERPARAMETERS - EXCLUDING APOE4

In [22]:
# LOAD PICKLED FEATURE_SET_7, IF EXISTS

for file in listdir(getcwd()+"/files"):
    if file.endswith("_feature_set_7.csv"):
        selector_feature_set_7 = pd.read_csv("files/"+file)
        
# SET HOW MANY REPETITION OF THE RFE 

rep = 10

if "selector_feature_set_7" not in globals():

    selector_feature_set_7 = pd.DataFrame(np.zeros((rep,len(feature_set_1))))
    
    for i in range(rep):
        
        # SET SEED
        np.random.seed(i)
    
        # CREATE AND FIT THE FILTER
        selector_feature_set_this = RFECV(estimator= sklearn.ensemble.RandomForestClassifier(n_estimators=2000), \
                                 cv=rkf, \
                                 scoring = "roc_auc",\
                                 n_jobs = -1)
        selector_feature_set_this.fit(X_train[:,feature_set_1], y=y_train)
        
        # FIND INDEX OF SELECTED FEATURES
        feature_this = [i for i in range(len(selector_feature_set_this.support_)) if selector_feature_set_this.support_[i] == True ]
        
        # ADD IT IN THE DATAFRAME
        selector_feature_set_7.iloc[i,feature_this] = 1
    
    # SAVE IT
    selector_feature_set_7.to_csv("selector_feature_set_7.csv", index = False)

# IDENTIFY WHICH FEATURES WERE SELECTED AT LEAST IN HALF OF THE REPETITIONS
pass_the_threshold = selector_feature_set_7.apply(sum) >= (rep/2)

# CREATE AN ARRAY WITH INDEXES OF FEATURES TO BE USED IN THE CURRENT FEATURE SET ACCORDING TO THE SELECTION STRATEGY
feature_set_7 = [feature_set_1[i] for i in range(selector_feature_set_7.shape[1]) if pass_the_threshold[i] == True ]
print([predictor_names[i] for i in feature_set_7])

['DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'LDELTOTAL', 'TRABSCOR', 'FAQ', 'PCA_1_ADAS', 'PCA_1_RAVLT_forgetting']


# TRAIN THE MODELS

### HELPER FUNCTIONS

In [23]:
# define log-uniform distribution for random search of svm rbf parameter search
class log_uniform():        
    def __init__(self, a=-1, b=0, base=10):
        self.loc = a
        self.scale = b - a
        self.base = base

    def rvs(self, size=None, random_state=None):
        uniform = sp.stats.uniform(loc=self.loc, scale=self.scale)
        if size is None:
            return np.power(self.base, uniform.rvs(random_state=random_state))
        else:
             return np.power(self.base, uniform.rvs(size=size, random_state=random_state))
            
# define the function that will be used to identify the threshold that provides the best balanced accuracy in the train dataset, with cv predictions 
def calculate_best_threshold_train_dataset(prediction_train_calibration, y_train_calibration = y_train):

    index_converters = np.where(y_train_calibration ==1)
    index_non_converters = np.where(y_train_calibration ==0)

    best_threshold = -10
    best_balanced_accuracy = -10

    for t in np.arange(0,1.0001,0.0001):
        sens = sum(prediction_train_calibration[index_converters] >= t)/prediction_train_calibration[index_converters].shape[0]
        spec = sum(prediction_train_calibration[index_non_converters] < t)/prediction_train_calibration[index_non_converters].shape[0]
    
        if best_balanced_accuracy <= ((sens+spec)/2):
            best_threshold = t
            best_balanced_accuracy = ((sens+spec)/2)
            
            # print(t)
            # print(best_balanced_accuracy)
    
    return best_threshold

# define the function that will be used to identify the thresholds that provide desired sensitivities in the train dataset, with cv predictions      
def calculate_parametrized_sensitivity_threshold_train_dataset(prediction_train_calibration, y_train_calibration = y_train, desired_sensitivity_levels = [0.95,0.9,0.85,0.8,0.75]):

    index_converters = np.where(y_train_calibration ==1)
    index_non_converters = np.where(y_train_calibration ==0)

    # best_threshold = -10
    # current_sensitivity_level = -10
    
    results_all = pd.DataFrame({"Threshold" : [], "Sensitivity":[], "Specificity":[]})
    results_final = pd.DataFrame({"Sensitivity_theoretical":desired_sensitivity_levels,"Threshold" : np.repeat(-10, len(desired_sensitivity_levels)), "Sensitivity":np.repeat(-10, len(desired_sensitivity_levels)), "Specificity":np.repeat(-10, len(desired_sensitivity_levels))})
    
    for t in np.arange(0,1.0001,0.0001):
        results_all = results_all.append(pd.Series({"Threshold":t,  \
                                                      "Sensitivity":(sum(prediction_train_calibration[index_converters] >= t)/prediction_train_calibration[index_converters].shape[0]), \
                                                      "Specificity":(sum(prediction_train_calibration[index_non_converters] < t)/prediction_train_calibration[index_non_converters].shape[0] ) \
                                                     }),\
                                           ignore_index=True)
        
    for i in desired_sensitivity_levels:
        index = np.where(  abs(i- results_all["Sensitivity"].values) == np.min(abs(i- results_all["Sensitivity"].values))   )[0]
        index = index[len(index)-1]
        results_final.loc[results_final.Sensitivity_theoretical == i, ["Threshold","Sensitivity","Specificity"]]  = results_all.iloc[index].values

    return results_final


### CREATE CROSS-VALIDATION ITERATION PARAMETERS

In [24]:
# REPEATED STRATIFIED 10-FOLD CV, REPEATED 10 TIMES.
n_fold = 10
n_repeats = 10

# SET SEED FOR REPRODUCIBILITY
cv_random_seed = 1

# DEFINE CV OBJECT
rkf = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=n_repeats, random_state=cv_random_seed)

# DEFINE THE NUMBER OF RANDOM SEARCHES TO PERFORM BEFORE BAYESIAN OPTIMIZATION SEARCH STARTS
n_random_combination = 50

# DEFINE THE TOTAL NUMBER OF THE BAYESIAN OPTIMIZATION SEARCH, INCLUDING THE INITIAL RANDOM SEARCHES
n_BO_combination = 50 + n_random_combination

# DEFINE AUROC AS THE METRIC TO BE OPTIMIZED IN THE HYPER-PARAMETER SEARCH
scorer_1 = make_scorer(roc_auc_score, needs_proba=True)
scorer_this = "roc_auc"

### DEFINE WHICH MODELS YOU WANT TO TRAIN

In [25]:
# names

models = ["EN_CATEGORICAL",\
          "svm_linear",\
          "svm_radial",\
          "svm_poly",\
          "knn",\
          "rf",\
          "MLP_1hiddenlayer_batch",\
          "MLP_2hiddenlayer_batch",\
          "MLP_1hiddenlayer_adam",\
          "MLP_2hiddenlayer_adam",\
          "gb"]

In [26]:
# create untrained models
EN_CATEGORICAL = SGDClassifier(loss= "log", penalty = "elasticnet") # no way to use the submodel trick with elasticnet for classification in sklearn...
svm_linear= sklearn.svm.SVC(kernel='linear', random_state=cv_random_seed, probability=True, max_iter=500)
svm_radial= sklearn.svm.SVC(kernel='rbf', random_state=cv_random_seed, probability=True, max_iter=500)
svm_poly= sklearn.svm.SVC(kernel='poly', random_state=cv_random_seed, probability=True, max_iter=500)
knn = sklearn.neighbors.KNeighborsClassifier()
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=2000)
gb = sklearn.ensemble.GradientBoostingClassifier(n_estimators=500)

# MLP classifiers need to be defined with a wrapper, due to the way hyperparameters are declaired in the class (which is not natively supported by the scikit-optimize API)

class MLP_1hiddenlayer_batch(BaseEstimator, ClassifierMixin):
    def __init__(self, layer1=10, activation="logistic", learning_rate_init = 0.01):
        self.layer1 = layer1
        self.activation = activation
        self.learning_rate_init = learning_rate_init
        self.classes_ = [0,1]
        
    def fit(self, X, y):
        model = MLPClassifier(
            hidden_layer_sizes=[self.layer1],\
            activation = self.activation,\
            solver = "adam", \
            learning_rate_init = self.learning_rate_init,\
            batch_size = 100000000,\
            max_iter = 2000,\
            random_state = cv_random_seed,\
            verbose = 0
        )
        
        model.fit(X, y)
        self.model = model
        return self

    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def score(self, X, y):
        return self.model.score(X, y)

class MLP_2hiddenlayer_batch(BaseEstimator, ClassifierMixin):
    def __init__(self, layer1=10, layer2=10, activation="logistic", learning_rate_init = 0.01):
        self.layer1 = layer1
        self.layer2 = layer2
        self.activation = activation
        self.learning_rate_init = learning_rate_init
        self.classes_ = [0,1]
        
    def fit(self, X, y):
        model = MLPClassifier(
            hidden_layer_sizes=[self.layer1, self.layer2],\
            activation = self.activation,\
            solver = "adam", \
            learning_rate_init = self.learning_rate_init,\
            batch_size = 100000000,\
            max_iter = 2000,\
            random_state = cv_random_seed,\
            verbose = 0
        )
        
        model.fit(X, y)
        self.model = model
        return self

    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def score(self, X, y):
        return self.model.score(X, y)

class MLP_1hiddenlayer_adam(BaseEstimator, ClassifierMixin):
    def __init__(self, layer1=10, activation="logistic", learning_rate_init = 0.01, batch_size = 100):
        self.layer1 = layer1
        self.activation = activation
        self.learning_rate_init = learning_rate_init
        self.batch_size = batch_size
        self.classes_ = [0,1]
        
    def fit(self, X, y):
        model = MLPClassifier(
            hidden_layer_sizes=[self.layer1],\
            activation = self.activation,\
            solver = "adam", \
            learning_rate_init = self.learning_rate_init,\
            batch_size = self.batch_size,\
            max_iter = 2000,\
            random_state = cv_random_seed,\
            verbose = 0
        )
        
        model.fit(X, y)
        self.model = model
        return self

    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def score(self, X, y):
        return self.model.score(X, y)
    

class MLP_2hiddenlayer_adam(BaseEstimator, ClassifierMixin):
    def __init__(self, layer1=10, layer2=10, activation="logistic", learning_rate_init = 0.01, batch_size = 100):
        self.layer1 = layer1
        self.layer2 = layer2
        self.activation = activation
        self.learning_rate_init = learning_rate_init
        self.batch_size = batch_size
        self.classes_ = [0,1]
        
    def fit(self, X, y):
        model = MLPClassifier(
            hidden_layer_sizes=[self.layer1, self.layer2],\
            activation = self.activation,\
            solver = "adam", \
            learning_rate_init = self.learning_rate_init,\
            batch_size = self.batch_size,\
            max_iter = 2000,\
            random_state = cv_random_seed,\
            verbose = 0
        )
        
        model.fit(X, y)
        self.model = model
        return self

    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def score(self, X, y):
        return self.model.score(X, y)

In [27]:
# hyperparameter search for each model

def dict_parameter_BO_EN_CATEGORICAL(features_this):
    return {'l1_ratio' : (1e-256, 1, 'uniform'),\
            'alpha' : (1e-6, 1e+6, 'log-uniform')}

def dict_parameter_BO_svm_linear(features_this):
    return {'C': (1e-3, 1e+3, 'log-uniform')}

def dict_parameter_BO_svm_radial(features_this):
    return {'C' : (1e-3, 1e+3, 'log-uniform'),  \
            'gamma': (1e-3, 1e+3, 'log-uniform')}

def dict_parameter_BO_svm_poly(features_this):
    return {'degree': (1,3),\
            'coef0': (-1e3, 1e3, 'uniform'),\
            'gamma': (1e-3, 1e+3, 'log-uniform')}

def dict_parameter_BO_knn(features_this):
    return {"n_neighbors" : (1,len(features_this)), \
           "weights" : ["uniform","distance"],
           "algorithm" : ["brute"],
           "p": (1, 4, 'uniform')}

def dict_parameter_BO_rf(features_this):
    return {"criterion" : ["gini", "entropy"],\
                "max_features" : (1, len(features_this))}

def dict_parameter_BO_MLP_1hiddenlayer_batch(features_this):
    return {'learning_rate_init' : (1e-6, 0.5, 'uniform'),\
            'activation': ['logistic', 'relu', 'tanh'],\
            'layer1' : (1, 100)}
              
def dict_parameter_BO_MLP_2hiddenlayer_batch(features_this):
    return {'layer1' : (1, 100),  \
            'layer2' : (1, 100),  \
            'activation': ['logistic', 'relu', 'tanh'], \
            'learning_rate_init' : (1e-6, 0.5, 'uniform')}

def dict_parameter_BO_MLP_1hiddenlayer_adam(features_this):
    return {'learning_rate_init' : (1e-6, 0.5, 'uniform'),\
            'activation': ['logistic', 'relu', 'tanh'],\
            'layer1' : (1, 100),\
            'batch_size' : (50,round(X_train.shape[0],0))}
              
def dict_parameter_BO_MLP_2hiddenlayer_adam(features_this):
    return {'layer1' : (1, 100),  \
            'layer2' : (1, 100),  \
            'activation': ['logistic', 'relu', 'tanh'], \
            'learning_rate_init' : (1e-6, 0.5, 'uniform'),\
            'batch_size' : (50,round(X_train.shape[0],0))}

def dict_parameter_BO_gb(features_this):
    return {'learning_rate' : (1e-3, 0.3, 'uniform'),\
            'max_depth' : (2, 6),\
            'min_samples_split' : (2, 4),\
            'subsample' : (0.8, 1, 'uniform'),\
            'max_features' : (None, "sqrt")}


### LOAD MODELS THAT HAVE ALREADY BEEN TRAINED AND PICKLED. 
### FOR THOSE, DON'T RUN AGAIN THE CODE THAT WOULD RE-TRAIN THEM 

In [28]:
# LOAD MODELS WHICH HAVE ALREADY BEEN TRAINED AND PICKLED

for file in listdir(getcwd()+"/trained_models"):
    if file.endswith(".pickle") and "_result_" in file:
        name_object = re.sub(".pickle","",file)
        globals()[name_object] = joblib.load("trained_models/"+file)

### TRAIN ALL MODELS, EXCLUDING LOGISTIC REGRESSION AND NAIVE-BAYES CLASSIFIERS

In [29]:
# FIRST IT WILL CHECK IF A CERTAIN MODELS ALREADY EXISTS. ONLY IF NOT, IT WILL TRAIN IT

for model in models:
    for feature_set in ["1","2","5","7"]:
        for search in ["BO"]:
            if search+"_result_"+model+"_feature_set_"+feature_set not in globals():
                
                print("                              ")
                print("                              ")
                print("-------------------------------------------------------")
                print("-------------------------------------------------------")
                print(search+"_result_"+model+"_feature_set_"+feature_set)
                print("                              ")
                
                # CREATE MODEL
                model_this = globals()[model]
                features_this = globals()["feature_set_"+feature_set]
                
                if search == "BO":
                    
                    if  model == "EN_CATEGORICAL" or "MLP" in model:
                        # as there will be some covergence warnings for very small level of alpha, let's kill them for now
                        # also I used a trick to make the MLP in normal full batch training, which results in a warning message 
                        warnings.filterwarnings("ignore")
                    
                    # MLP models have been cretaed with a wrapper, so the need to be called slightly differently 
                    if "MLP" in model:
                    
                        search_this = BayesSearchCV(estimator=model_this(),\
                                                search_spaces = globals()["dict_parameter_"+search+"_"+model](features_this),\
                                                optimizer_kwargs = {"base_estimator" : "GP", \
                                                                    "acq_func" : "gp_hedge",\
                                                                    "n_initial_points" : n_random_combination,\
                                                                    "random_state" : cv_random_seed},\
                                                cv = rkf,\
                                                verbose=1,\
                                                n_iter=n_BO_combination,\
                                                n_jobs=-1,\
                                                scoring= scorer_this,\
                                                random_state=cv_random_seed,\
                                                iid=False,\
                                                return_train_score=True)
                        
                    else:
                        search_this = BayesSearchCV(estimator=model_this,\
                                                search_spaces = globals()["dict_parameter_"+search+"_"+model](features_this),\
                                                optimizer_kwargs = {"base_estimator" : "GP", \
                                                                    "acq_func" : "gp_hedge",\
                                                                    "n_initial_points" : n_random_combination,\
                                                                    "random_state" : cv_random_seed},\
                                                cv = rkf,\
                                                verbose=0,\
                                                n_iter=n_BO_combination,\
                                                n_jobs=-1,\
                                                scoring= scorer_this,\
                                                random_state=cv_random_seed,\
                                                iid=False,\
                                                return_train_score=True)
                    
                    
                    def on_step(optim_result):
                        print(" ")
                        print("TRAIN N° "+str(len(search_this.cv_results_['mean_fit_time'])))
                        print("BEST SCORE: %s" % str(search_this.best_score_))
                        print(" ")
                        print("best hyperparameters: %s" % str(search_this.best_params_))
                        print(" ")
                        print("current hyperparameters: %s" % search_this.cv_results_['params'][-1])
                        print(" ")
                        print("-------------------------------------------------------")
                        if search_this.best_score_ >= 1:
                            print('Interrupting!')
                            
                    # 
                    search_this.fit(X=X_train[:,features_this], y=y_train, callback=on_step)
                        
                    if model == "EN_CATEGORICAL" or "MLP" in model:
                        # bring back to the default print of warning messages
                        warnings.filterwarnings("default")
                        
                else:
                    print("Error: you have requested an hyperparameter search that you didn't properly defined")
                    
                globals()[search+"_result_"+model+"_feature_set_"+feature_set] = search_this
                
                clear_output()
                
                # PRINT THE BEST CROSS-VALIDATED PERFORMANCE
                print(" ")
                print("The model "+search+"_"+model+"_feature_set_"+feature_set+" has been trained.")
                print(" ")
                print("BEST CV PERFORMANCE: "+ str(search_this.best_score_))

                # DUMP THE MODEL
                joblib.dump(search_this, "trained_models/"+search+"_result_"+model+"_feature_set_"+feature_set+".pickle")

### TRAIN LOGISTIC REGRESSION MODELS

In [30]:
# FIRST IT WILL CHECK IF A CERTAIN MODELS ALREADY EXISTS. ONLY IF NOT, IT WILL TRAIN IT

for feature_set in ["1","2","5","7"]:
        
            if "LOGISTIC_result_feature_set_"+feature_set not in globals():
                
                print("                              ")
                print("                              ")
                print("-------------------------")
                print("-------------------------")
                print("LOGISTIC_result_feature_set_"+feature_set)
                print("                              ")
                
                
                # CREATE MODEL
                features_this = globals()["feature_set_"+feature_set]
                
                # NB: A PURE UNREGULARIZED LOGISTIC REGRESSION IS NOT IMPLEMENTED IN SKLEARN !?!. I ARTIFICIALLY MADE IT USING AN "ALMOST INFINITE" VALUE FOR C
                search_this =  GridSearchCV(estimator=LogisticRegression(),\
                                            param_grid = dict(penalty = ['l2'], C = [sys.maxsize]),\
                                            cv=rkf,\
                                            verbose = 1,\
                                            n_jobs=-1,\
                                            scoring = scorer_this,\
                                            iid=False,\
                                            return_train_score=True)
                search_this.fit(X=X_train[:,features_this], y=y_train)
                
                globals()["LOGISTIC_result_feature_set_"+feature_set] = search_this
                
                # PRINT THE BEST CROSS-VALIDATED PERFORMANCE
                print(" ")
                print("The model LOGISTIC_feature_set_"+feature_set+" has been trained.")
                print(" ")
                print("BEST CV PERFORMANCE: "+ str(search_this.best_score_))

                # DUMP THE MODEL
                joblib.dump(search_this, "trained_models/LOGISTIC_result_feature_set_"+feature_set+".pickle")

### TRAIN NAIVE-BAYES MODELS

In [31]:
# FIRST IT WILL CHECK IF A CERTAIN MODELS ALREADY EXISTS. ONLY IF NOT, IT WILL TRAIN IT

for feature_set in ["1","2","5","7"]:
        
            if "NB_result_feature_set_"+feature_set not in globals():
                
                print("                              ")
                print("                              ")
                print("-------------------------")
                print("-------------------------")
                print("NB_result_feature_set_"+feature_set)
                print("                              ")
                
                
                # CREATE MODEL
                features_this = globals()["feature_set_"+feature_set]
                
                # NB: A PURE UNREGULARIZED LOGISTIC REGRESSION IS NOT IMPLEMENTED IN SKLEARN !?!. I ARTIFICIALLY MADE IT USING AN "ALMOST INFINITE" VALUE FOR C
                search_this =  GridSearchCV(estimator=GaussianNB(),\
                                            param_grid = dict(priors = [None]),\
                                            cv=rkf,\
                                            verbose = 1,\
                                            n_jobs=-1,\
                                            scoring = scorer_this,\
                                            iid=False,\
                                            return_train_score=True)
                search_this.fit(X=X_train[:,features_this], y=y_train)
                
                globals()["NB_result_feature_set_"+feature_set] = search_this
                
                # PRINT THE BEST CROSS-VALIDATED PERFORMANCE
                print(" ")
                print("The model NB_feature_set_"+feature_set+" has been trained.")
                print(" ")
                print("BEST CV PERFORMANCE: "+ str(search_this.best_score_))

                # DUMP THE MODEL
                joblib.dump(search_this, "trained_models/NB_result_feature_set_"+feature_set+".pickle")

# CROSS-VALIDATED RESULTS

In [32]:
cv_results_array = pd.DataFrame(np.array(["",0,"",0]).reshape((2,2)), columns= ["MODEL","cv_score"]   )

models = [i for i in globals().keys() if ("_result_" in i) and (("_1" in i) or ("_2" in i) or ("_5" in i) or ("_7" in i))]

counter = 0
for model in models:
    
    if "cv_predictions" not in model:    
        cv_results_array.loc[counter] = [model,globals()[model].best_score_]
        counter += 1
        
cv_results_array = cv_results_array.sort_values(by="cv_score", ascending=False)        
cv_results_array.index = list(range(cv_results_array.shape[0]))

display(cv_results_array)


,MODEL,cv_score
0,BO_result_MLP_1hiddenlayer_adam_feature_set_7,0.901503
1,BO_result_MLP_2hiddenlayer_adam_feature_set_7,0.898309
2,BO_result_MLP_1hiddenlayer_adam_feature_set_1,0.897762
3,BO_result_MLP_2hiddenlayer_batch_feature_set_1,0.894108
4,BO_result_gb_feature_set_5,0.893861
5,BO_result_MLP_1hiddenlayer_adam_feature_set_2,0.893828
6,BO_result_rf_feature_set_7,0.892566
7,BO_result_MLP_1hiddenlayer_batch_feature_set_5,0.892191
8,BO_result_rf_feature_set_1,0.89219
9,BO_result_gb_feature_set_2,0.89217


# LOAD PREVIOUSLY SAVED CROSS-VALIDATED PREDICTIONS OF THE MODEL + CREATE AND SAVE THOSE THAT HAVE NOT BEEN CREATED YET

### A DIFFERENT NON-REPEATED STRATIFIED 10-FOLD CV SCHEME IS USED TO GENERATE THE PREDICTIONS. IN SKLEARN, YOU CANNOT GENERATE PREDICTION VIA CV FOR REPEATED SCHEMES!

In [33]:
# LOAD PREVIOUSLY SAVED CV PREDICTIONS
for file in listdir(getcwd()+"/trained_models"):
    if file.endswith(".pickle") and "cv_predictions" in file:
        name_object = re.sub(".pickle","",file)
        globals()[name_object] = joblib.load("trained_models/"+file)
        
# CREATE AND SAVE THOSE WHICH HAVE NOT BEEN CREATED YET
for model in cv_results_array["MODEL"]:
    if "cv_predictions_"+model not in globals():
        feature_this = "feature_set_"+str(model[-1])
        if feature_this == "feature_set_0":
            feature_this = "feature_set_10"
        X_train_this = X_train[:,globals()[feature_this]]
        warnings.filterwarnings("ignore")
        
        # it is impossible to use a repeated cv scheme with cross_val_predict. Thus, changed it to single 10-fold cv
        prediction_cv_this = sklearn.model_selection.cross_val_predict(estimator = globals()[model].best_estimator_,\
                                                                       X = X_train_this,\
                                                                       y = y_train,\
                                                                       cv = StratifiedKFold(n_splits=10,random_state=1),
                                                                       method = "predict_proba")[:,1]
        warnings.filterwarnings("default")
        globals()["cv_predictions_"+model] = prediction_cv_this
        joblib.dump(prediction_cv_this,getcwd()+"/trained_models/cv_predictions_"+model+".pickle")

# WEIGHTED RANK AVERAGE ENSAMBLING

### ALL MODELS WILL BE AVERAGED

In [34]:
chosen_models_index = [i for i in range(cv_results_array.shape[0])]

### CREATE PREDICTIONS FROM CV PREDICTIONS IN THE TRAIN SET

In [35]:
if "prediction_train" in globals().keys():
    del prediction_train

for model in list(cv_results_array["MODEL"].iloc[chosen_models_index]):
    prediction_this = globals()["cv_predictions_"+model]
    # prediction_this = np.log(prediction_this/(1- prediction_this))
    prediction_this = np.array(prediction_this).reshape((X_train.shape[0],1))
    if "prediction_train" not in globals().keys():
        prediction_train = prediction_this
    else:
        prediction_train = np.append(prediction_train, prediction_this, axis=1)

In [36]:
# calculate rank predictions train
prediction_train_rank = pd.DataFrame(prediction_train.copy()).rank(axis = 0)/X_train.shape[0]

# calculate weights according to cv performance
weights_cv = np.array(cv_results_array.cv_score.iloc[chosen_models_index])

# calculate weighted average rank predictions train
prediction_train_rank_average = np.array(prediction_train_rank.apply(lambda x: np.average(x, weights=weights_cv), axis=1))

### CREATE PREDICTION OF THE TEST SET

In [37]:
if "prediction_test" in globals().keys():
    del prediction_test

for model in list(cv_results_array["MODEL"].iloc[chosen_models_index]):
    feature = "feature_set_"+str(model[-1])
    if feature == "feature_set_0":
        feature = "feature_set_10"
    prediction_this = globals()[model].predict_proba(X=X_test[:,globals()[feature]])[:,1]
    prediction_this = np.array(prediction_this).reshape((X_test.shape[0],1))
    if "prediction_test" not in globals().keys():
        prediction_test = prediction_this
    else:
        prediction_test = np.append(prediction_test, prediction_this, axis=1)

In [38]:
# calculate rank predictions test according to train set
prediction_test_rank = pd.DataFrame(prediction_test.copy()).rank(axis = 0)/X_test.shape[0]

for j in range(prediction_test_rank.shape[1]):
    for i in range(prediction_test_rank.shape[0]):
        new_value_index = np.where(np.abs(prediction_test[i,j] - prediction_train[:,j]) == np.min(np.abs(prediction_test[i,j] - prediction_train[:,j])))[0][0]
        new_value = prediction_train_rank.iloc[new_value_index,j]
        prediction_test_rank.iloc[i,j] = new_value

# calculate weighted average rank predictions train
prediction_test_rank_average = np.array(prediction_test_rank.apply(lambda x: np.average(x, weights=weights_cv), axis=1))

In [39]:
print("### TESTING AVERAGE RANK")
print("__________________________________________________")
print("     ")

prediction_BEST_MODEL = prediction_test_rank_average
print("TEST AUCROC SCORE: "+str(roc_auc_score(y_true = y_test, y_score=prediction_BEST_MODEL)))
print(" ")

np.savetxt("files/roc_test_wra.csv",\
           np.array(roc_auc_score(y_true = y_test, y_score=prediction_BEST_MODEL)).reshape((1,1)),\
           delimiter="#")

t = calculate_best_threshold_train_dataset(prediction_train_calibration=prediction_train_rank_average, y_train_calibration = y_train)
np.savetxt("files/t_best_balanaced_accuracy_wra.csv",[t],delimiter="#")

# EXPORT TEST PREDICTIONS AND REFERENCES

np.savetxt("files/probabilistic_prediction_test_wra.csv", prediction_BEST_MODEL, delimiter=",")
np.savetxt("files/categorical_outcome_test_wra.csv", \
           [1 if i >= t else 0 for i in prediction_BEST_MODEL],\
           delimiter=",")

print("THRESHOLD (MAXIMIZE BALANCED ACCURACY IN THE TRAIN SET): "+ str(t))
print(" ")

index_converters_test = np.where(y_test ==1)
index_non_converters_test = np.where(y_test ==0)

sens_test = sum(prediction_BEST_MODEL[index_converters_test] >= t) / prediction_BEST_MODEL[index_converters_test].shape[0]
spec_test = sum(prediction_BEST_MODEL[index_non_converters_test] < t) / prediction_BEST_MODEL[index_non_converters_test].shape[0]
ppv_test = sum(prediction_BEST_MODEL[index_converters_test] >= t) / sum(prediction_BEST_MODEL >= t)
npv_test = sum(prediction_BEST_MODEL[index_non_converters_test] < t) / sum(prediction_BEST_MODEL < t)

print("BALANCED ACCURACY:                   "+ str((sens_test+spec_test)/2))
print("F1-SCORE:                            "+ str(2*ppv_test*sens_test/(ppv_test+sens_test)))
print("SENSITIVITY/RECALL:                  "+ str(sens_test))
print("SPECIFICITY:                         "+ str(spec_test))
print("POSITIVE PREDICTIVE VALUE/PRECISION: "+ str(ppv_test))
print("NEGATIVE PREDICTIVE VALUE:           "+ str(npv_test))
print("     ")
print("_______________________________________________")
print("     ")

# MOVING SENSITIVITY LEVELS

t_moving_results = calculate_parametrized_sensitivity_threshold_train_dataset(prediction_train_calibration=prediction_train_rank_average, y_train_calibration = y_train, desired_sensitivity_levels = [1.0,0.99,0.975,0.95,0.9,0.85,0.8,0.75])

np.savetxt("files/t_moving_results_wra.csv", t_moving_results, delimiter=",")
for i in range(t_moving_results.shape[0]):
    np.savetxt("files/categor_moving_sensitivity_outcome_test_"+str(t_moving_results.iloc[i,0])+"_wra.csv", \
           [1 if pred >= t_moving_results.iloc[i,1] else 0 for pred in prediction_BEST_MODEL],\
           delimiter=",")

### TESTING AVERAGE RANK
__________________________________________________
     
TEST AUCROC SCORE: 0.859706959707
 
THRESHOLD (MAXIMIZE BALANCED ACCURACY IN THE TRAIN SET): 0.5728
 
BALANCED ACCURACY:                   0.745970695971
F1-SCORE:                            0.68085106383
SENSITIVITY/RECALL:                  0.820512820513
SPECIFICITY:                         0.671428571429
POSITIVE PREDICTIVE VALUE/PRECISION: 0.581818181818
NEGATIVE PREDICTIVE VALUE:           0.87037037037
     
_______________________________________________
     


# UNIVARIATE FEATURE IMPORTANCE

In [40]:
# LOAD FEATURE IMPORTANCE WHICH HAVE ALREADY BEEN CREATED AND PICKLED

for file in listdir(getcwd()+"/univariate_feature_importance"):
    if file.endswith(".pickle") and "UNIVARIATE_" in file:
        name_object = re.sub(".pickle","",file)
        globals()[name_object] = joblib.load("univariate_feature_importance/"+file)


# CREATE DATAFRAME OF RESULTS
UNIVARIATE_FEATURE_IMPORTANCE = pd.DataFrame(np.array(["",0,"",0]).reshape((2,2)), columns= ["FEATURE","AUC_cv_score"]   )

# # TRAIN THE SINGLE FEATURE MODELS, ONLY FOR THOSE THAT WERE NOT TRAINED BEFORE

for feature_this in feature_set_1:
    
    if "UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this]) not in globals():
        
        search_this =  GridSearchCV(estimator=LogisticRegression(),\
                                            param_grid = dict(penalty = ['l2'], C = [sys.maxsize]),\
                                            cv=rkf,\
                                            verbose = 1,\
                                            n_jobs=-1,\
                                            scoring = scorer_this,\
                                            iid=False,\
                                            return_train_score=True)
                
        search_this.fit(X=X_train[:,feature_this].reshape((X_train.shape[0],1)), y=y_train)
    
        # SAVE IT IN THE GLOBAL ENVIRONMENT
        globals()["UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this])] = search_this
        
        # DUMP THE MODEL
        joblib.dump(search_this, "univariate_feature_importance/UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this])+".pickle")
    
    if "prediction_test_UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this]) not in globals():
        search_this =  GridSearchCV(estimator=LogisticRegression(),\
                                            param_grid = dict(penalty = ['l2'], C = [sys.maxsize]),\
                                            cv=rkf,\
                                            verbose = 1,\
                                            n_jobs=-1,\
                                            scoring = scorer_this,\
                                            iid=False,\
                                            return_train_score=True)
                
        search_this.fit(X=X_train[:,feature_this].reshape((X_train.shape[0],1)), y=y_train)
        
        # GENERATE TEST PREDICTIONS
        prediction_this = search_this.predict_proba(X=X_test[:,feature_this].reshape((X_test.shape[0],1)))[:,1]
    
        # SAVE THE PREDICTIONS
        np.savetxt(getcwd()+"/univariate_feature_importance/prediction_test_UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this])+"_"+str(fold)+".csv", prediction_this, delimiter=",")
    
    # UPDATE THE DATAFRAME OF REULTS
    UNIVARIATE_FEATURE_IMPORTANCE.loc[feature_this] = [predictor_names[feature_this],\
                                                       globals()["UNIVARIATE_FEATURE_IMPORTANCE_"+str(predictor_names[feature_this])].best_score_]
    
# SAVE RESULTS

UNIVARIATE_FEATURE_IMPORTANCE.to_excel("UNIVARIATE_FEATURE_IMPORTANCE_fold_"+str(fold)+".xlsx")

Fitting 100 folds for each of 1 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   5 out of 100 | elapsed:    0.1s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting 100 folds for each of 1 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   5 out of 100 | elapsed:    0.1s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


Fitting 100 folds for each of 1 candidates, totalling 100 fits


...

[Parallel(n_jobs=-1)]: Done   5 out of 100 | elapsed:    0.1s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


Fitting 100 folds for each of 1 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   5 out of 100 | elapsed:    0.1s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


Fitting 100 folds for each of 1 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   5 out of 100 | elapsed:    0.1s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


# SAVE CV RESULTS

In [41]:
cv_results_array = cv_results_array.sort_values(by="MODEL", ascending=False)        
        
cv_results_array["technique"] = ""
for i in range(cv_results_array.shape[0]):
    if "NB" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Naïve Bayes"
    elif "LOGISTIC" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Logistic Regression"
    elif "svm_radial" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Support Vector Machine, polynomial kernel"
    elif "svm_poly" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Support Vector Machine, rbf kernel"
    elif "svm_linear" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Support Vector Machine, linear kernel"
    elif "rf" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Random Forest"
    elif "knn" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "k-Nearest Neighbor"
    elif "gb" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Gradient Boosting Machine"
    elif "MLP_1hiddenlayer_batch" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Multi-layer Perceptrion, 1 hidden layer and batch learning"
    elif "MLP_2hiddenlayer_batch" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Multi-layer Perceptrion, 2 hidden layers and batch learning"
    elif "MLP_1hiddenlayer_adam" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Multi-layer Perceptrion, 1 hidden layer and adam learning"
    elif "MLP_2hiddenlayer_adam" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Multi-layer Perceptrion, 2 hidden layers and adam learning"
    elif "EN_CATEGORICAL" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["technique"].iloc[i] = "Elastic Net"

cv_results_array["feature"] = 0
for i in range(cv_results_array.shape[0]):
    if "feature_set_1" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["feature"].iloc[i] = 1
    elif "feature_set_2" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["feature"].iloc[i] = 3
    elif "feature_set_5" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["feature"].iloc[i] = 5
    elif "feature_set_7" in cv_results_array["MODEL"].iloc[i]:
        cv_results_array["feature"].iloc[i] = 7

cv_results_array = cv_results_array.sort_values(by=["feature","technique"], ascending=False)        

cv_results_array.to_excel("cv_results_fold_"+str(fold)+".xlsx")

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
